In [61]:
import numpy as np
import pandas as pd
import pickle
import re
import string

In [62]:
with open('../static/Model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [63]:
vocab = pd.read_csv("../static/Model/vocabulary.txt",header=None)
tokens = vocab[0].tolist()

In [64]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [65]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [66]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [67]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=["tweet"])
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["tweet"] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data["tweet"] = data["tweet"].apply(remove_punctuations)
    data["tweet"] = data['tweet'].str.replace(r'\d+', '', regex=True)
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data["tweet"]

In [68]:
def vectorizer(ds, vocabulary):
    vectorized_list = []

    for sentence in ds:
        sentence_list=np.zeros(len(vocabulary))

        for i in range (len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_list[i] = 1

        vectorized_list.append(sentence_list)

    vectorized_list_new = np.asarray(vectorized_list, dtype=np.float32)

    return vectorized_list_new

In [70]:
def predicted_val(txt):
    preprocessed_text = preprocessing(txt)
    vectorized_text = vectorizer(preprocessed_text, tokens)
    prediction = model.predict(vectorized_text)
    return(prediction)

In [75]:
def predicted_val(txt):
    preprocessed_text = preprocessing(txt)
    vectorized_text = vectorizer(preprocessed_text, tokens)
    prediction = model.predict(vectorized_text)
    
    if prediction ==1:
            return 'Negative'
    else:
            return 'Positive'

In [76]:
txt1 = "Bad product. I hate it."

In [77]:
predicted_val(txt1)

'Negative'